# Neural Network for racetime prediction

### Define a logger for logging experiments configurations and results

In [ ]:
import json
import os
from datetime import datetime
from pytorch_lightning.loggers import Logger
from pytorch_lightning.utilities.rank_zero import rank_zero_only

class F1NeuralNetworkExperimentsLogger(Logger):
    def __init__(self):
        super().__init__()
        self.metrics = []
        self.logs = {}
        self._version = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    @property
    def name(self):
        return "F1NeuralNetworkExperimentsLogger"

    @property
    def version(self):
        return self._version
    
    @rank_zero_only
    def log_model_architecture(self, model):
        def layer_to_dict(layer):
            """Convert a PyTorch layer to a structured dictionary."""
            return {
                'type': layer.__class__.__name__,
                'parameters': {
                    name: p.shape if hasattr(p, 'shape') else str(p)
                    for name, p in layer.named_parameters(recurse=False)
                },
                'submodules': [layer_to_dict(sub) for sub in layer.children()]
            }

        architecture_dict = layer_to_dict(model)
        self.logs['model_architecture'] = architecture_dict


    @rank_zero_only
    def log_hyperparams(self, params):
        self.logs['hyperparameters'] = {k: str(v) for k, v in params.items()}

    @rank_zero_only
    def log_metrics(self, metrics, step):
        self.metrics.append((step, metrics))
        if 'metrics' not in self.logs:
            self.logs['metrics'] = []
        self.logs['metrics'].append({'step': step, 'metrics': {k: float(v) for k, v in metrics.items()}})

    @rank_zero_only
    def log_overall_test_loss(self, test_loss):
        self.logs['overall_test_loss'] = test_loss

    @rank_zero_only
    def log_used_features(self, features):
        self.logs['used_features'] = features

    @rank_zero_only
    def log_optimization_strategy(self, optimizer, scheduler):
        optimizer_str = str(optimizer)
        scheduler_str = str(scheduler)
        self.logs['optimizer'] = optimizer_str
        self.logs['scheduler'] = scheduler_str

    @rank_zero_only
    def save(self):
        directory = os.path.join(self.name, self.version)
        if not os.path.exists(directory):
            os.makedirs(directory)
        with open(os.path.join(directory, "logs.json"), "w") as f:
            json.dump(self.logs, f, indent=4)

    @rank_zero_only
    def finalize(self, status):
        self.save()

logger = F1NeuralNetworkExperimentsLogger()

### Preprocess and load the features and labels for regression

In [ ]:
from data_preparation import load_and_preprocess_data, prepare_regression_data, split_data_by_race
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler

class F1Dataset(Dataset):
    def __init__(self, X, y, scaler=None):
        super().__init__()
        self.X = X.reset_index(drop=True)
        print(self.X.columns)
        self.y = y.reset_index(drop=True)
        self.scaler = scaler
        if self.scaler:
            self.scaler.fit(self.X)
            self.X = pd.DataFrame(self.scaler.fit_transform(self.X), columns=self.X.columns)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = torch.tensor(self.X.iloc[idx].values, dtype=torch.float32)  # Access by iloc
        y = torch.tensor(self.y.iloc[idx], dtype=torch.float32)  # Access by iloc
        return X, y

train_df, test_df = split_data_by_race(load_and_preprocess_data())
train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42)

X_train, y_train = prepare_regression_data(train_df)
X_val, y_val = prepare_regression_data(val_df)
X_test, y_test = prepare_regression_data(test_df)

N_FEATURES = X_train.shape[1]
print(f"Number of features: {N_FEATURES}")


scaler = StandardScaler()

train_dataset = F1Dataset(X_train, y_train, scaler=scaler)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = F1Dataset(X_val, y_val, scaler=scaler)
val_dataloader = DataLoader(val_dataset, batch_size=32)
test_dataset = F1Dataset(X_test, y_test, scaler=scaler)
test_dataloader = DataLoader(test_dataset, batch_size=32)

train_dataset.__getitem__(0)

### Model

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import matplotlib.pyplot as plt

class LaptimePredicionModel(pl.LightningModule):
    def __init__(self, lr=0.005, loss_fn=nn.MSELoss, optimizer=torch.optim.AdamW, layer_config=(2, 64)):
        super().__init__()
        self.save_hyperparameters(logger=True)
        self.predictions = []
        self.actuals = []
        self.train_losses = []
        self.val_losses = []
        self.test_losses = []
        self.lr = lr
        self.INPUT_DIM = N_FEATURES
        self.OUTPUT_DIM = 1

        # Get the number of layers and neurons per layer from layer_config
        num_layers, neurons_per_layer = layer_config
        
        # Build the model dynamically
        layers = []
        
        in_features = self.INPUT_DIM
        layers.append(nn.Linear(in_features, neurons_per_layer))
        for i in range(num_layers):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(nn.ReLU())
            in_features = neurons_per_layer
        
        # Final output layer
        layers.append(nn.Linear(neurons_per_layer, self.OUTPUT_DIM))

        self.model = nn.Sequential(*layers)
        self.loss_fn = loss_fn() if callable(loss_fn) else loss_fn
        self.optimizer = optimizer(self.model.parameters(), lr=self.lr)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x).squeeze(-1)
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.train_losses.append(loss.item())
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x).squeeze(-1)
        test_loss_fn = nn.MSELoss()
        loss = torch.sqrt(test_loss_fn(y_hat, y))
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.val_losses.append(loss.item())
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x).squeeze(-1)
        test_loss_fn = nn.MSELoss()
        loss = torch.sqrt(test_loss_fn(y_hat, y))
        self.predictions.extend(y_hat.cpu().numpy())
        self.actuals.extend(y.cpu().numpy())
        self.test_losses.append(loss.item())
        return loss

    def on_test_epoch_end(self):
        avg_test_loss = sum(self.test_losses) / len(self.test_losses)
        print(f"Average Test Loss: {avg_test_loss}")
        
        # Set the background color to black
        plt.figure(figsize=(8, 6), facecolor='black')
        ax = plt.gca()  # Get the current axis
        ax.set_facecolor('black')  # Set the plot area background color to black

        # Scatter plot with blue points and blue edges
        plt.scatter(self.actuals, self.predictions, alpha=0.6, color='b', edgecolor='b')  

        # Customize axis labels, title, and grid
        plt.xlabel('Actual Values', color='w')  # White axis label
        plt.ylabel('Predicted Values', color='w')  # White axis label
        plt.title('Correlation between Actual and Predicted Values', color='w')  # White title
        plt.grid(True, color='w')  # White grid lines
        
        # Customize the spines (axes borders) to have black background behind labels
        ax.spines['top'].set_color('w')
        ax.spines['right'].set_color('w')
        ax.spines['left'].set_color('w')
        ax.spines['bottom'].set_color('w')

        # Set the ticks and their labels to white
        ax.tick_params(axis='both', colors='w')

        plt.show()
        
        self.plot_loss_curve()

    def on_train_epoch_end(self):
        self.train_losses.clear()

    def configure_optimizers(self):
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            self.optimizer,
            max_lr=self.hparams.lr,
            steps_per_epoch=100,
            epochs=10,
            anneal_strategy='cos',
        )
        return {
            'optimizer': self.optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

    def plot_loss_curve(self):
        # Set the background color to black
        plt.figure(figsize=(10, 5), facecolor='black')
        ax = plt.gca()  # Get the current axis
        ax.set_facecolor('black')  # Set the plot area background color to black

        if self.val_losses:
            plt.plot(self.val_losses, label='Validation Loss', color='b')  # Blue line for loss curve

        # Customize axis labels, title, and grid
        plt.xlabel('Epoch', color='w')  # White axis label
        plt.ylabel('Loss', color='w')  # White axis label
        plt.title('Loss Curve', color='w')  # White title
        plt.legend(frameon=False, loc='best', facecolor='black', edgecolor='w', labelcolor='w')  # White text for legend
        plt.grid(True, color='w')  # White grid lines

        # Customize the spines (axes borders) to have black background behind labels
        ax.spines['top'].set_color('w')
        ax.spines['right'].set_color('w')
        ax.spines['left'].set_color('w')
        ax.spines['bottom'].set_color('w')

        # Set the ticks and their labels to white
        ax.tick_params(axis='both', colors='w')

        plt.show()

### Configure callback functions

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=3,          
    verbose=True,
    mode="min"           
)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",     
    filename="best_model-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,             
    mode="min",                  
    verbose=True
)

### Determine optimal amount of model layers and neurons per layer

In [ ]:
"""import pytorch_lightning as pl
import optuna
import csv
import os

def objective(trial):
    # Hyperparameters to tune
    num_layers = trial.suggest_int('num_layers', 2, 6)
    neurons_per_layer = trial.suggest_int('neurons_per_layer', 32, 128)

    # Initialize the model with sampled hyperparameters
    model = LaptimePredicionModel(layer_config=(num_layers, neurons_per_layer))

    # Trainer setup
    trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=[early_stopping, checkpoint_callback])

    # Train the model
    trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

    # Calculate the average validation loss
    avg_val_loss = sum(model.val_losses) / len(model.val_losses)
    print(f"Test Loss for {num_layers} layers, {neurons_per_layer} neurons per layer: {avg_val_loss:.4f}")

    # Log results to CSV
    log_complexity_results_to_csv('neural_net_hyperparam_evaluation/complexity_hyperparams.csv', num_layers, neurons_per_layer, avg_val_loss)

    return avg_val_loss


def log_complexity_results_to_csv(filename, num_layers, neurons_per_layer, avg_val_loss):
    # Check if the file exists
    file_exists = os.path.isfile(filename)

    # Open the file in append mode
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header only if the file is new
        if not file_exists:
            writer.writerow(['layers', 'neurons_per_layer', 'loss'])
        
        # Write the trial's results
        writer.writerow([num_layers, neurons_per_layer, avg_val_loss])


def optimize_hyperparameters():
    # Create a study to minimize the validation loss
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

    # Print the best hyperparameters and their corresponding loss
    print("Best Hyperparameters: ", study.best_params)
    print("Best Value (Test Loss): ", study.best_value)


# Run the optimization process
optimize_hyperparameters()"""

### Determine the optimal learning rate, optimizer and loss function

In [ ]:
"""import optuna
import csv
import os

# Define the objective function for Optuna
def objective(trial):
    # Expanded learning rate range
    lr = trial.suggest_loguniform('lr', 1e-6, 1)  # Learning rate (log scale, wider range)
    
    # Choose optimizer: Adam, AdamW, or SGD
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'AdamW', 'SGD'])
    if optimizer_name == 'Adam':
        optimizer = torch.optim.Adam
    elif optimizer_name == 'AdamW':
        optimizer = torch.optim.AdamW
    else:
        optimizer = torch.optim.SGD

    # Choose loss function: L1Loss or MSELoss
    loss_fn_name = trial.suggest_categorical('loss_fn', ['L1Loss', 'MSELoss'])
    if loss_fn_name == 'L1Loss':
        loss_fn = nn.L1Loss
    else:
        loss_fn = nn.MSELoss

    # Initialize the model with the fixed architecture and selected loss function
    model = LaptimePredicionModel(lr=lr, loss_fn=loss_fn, optimizer=optimizer)
    
    trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=[early_stopping, checkpoint_callback])

    trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

    # Calculate average test loss
    avg_val_loss = sum(model.val_losses) / len(model.val_losses)
    print(f"Test Loss for Optimizer: {optimizer_name}, Loss function: {loss_fn_name}, "
          f"Learning rate: {lr:.6f}: {avg_val_loss:.4f}")

    # Log results to CSV
    log_learning_results_to_csv('neural_net_hyperparam_evaluation/learning_hyperparams.csv', lr, optimizer_name, loss_fn_name, avg_val_loss)

    return avg_val_loss  # We aim to minimize the test loss


# Function to log trial results to a CSV file
def log_learning_results_to_csv(filename, lr, optimizer_name, loss_fn_name, avg_val_loss):
    # Check if the file exists
    file_exists = os.path.isfile(filename)

    # Open the file in append mode
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header only if the file is new
        if not file_exists:
            writer.writerow(['learning_rate', 'optimizer', 'loss_fn', 'loss'])
        
        # Write the trial's results
        writer.writerow([lr, optimizer_name, loss_fn_name, avg_val_loss])


# Function to run the optimization process
def optimize_hyperparameters():
    # Create an Optuna study to minimize the test loss
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)  # Run 10 trials

    # Print the best hyperparameters found by Optuna
    print("Best Hyperparameters: ", study.best_params)
    print("Best Test Loss: ", study.best_value)

# Start the optimization process
optimize_hyperparameters()"""

### Final Training with best configuration

In [ ]:
model = LaptimePredicionModel()
trainer = pl.Trainer(
    callbacks=[early_stopping, checkpoint_callback],
    log_every_n_steps=10,
    max_epochs=1000,
)

trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

In [ ]:
trainer.test(model, val_dataloader)

### Final Evaluation on test data

In [ ]:
trainer.test(model=model, dataloaders=test_dataloader)